In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
%run "/eReport/PkgRcd/Cálculo de Provisiones"

In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np

class PrcGeneraPlanoRcd():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d')
      self.tabla_destino = parametros['tabla_destino']
      self.tabla_destino_2 = parametros['tabla_destino_2']
      self.empresa = parametros['empresa']
      self.empresa_str = str(self.empresa)
      self.ruta_destino = '/dbfs/mnt/ereportinterfaz/outseir/'
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
  
  def generarPlanoRcd(self):
      
      ###### ------ Rcd cabecera ------ ######
      #Obteniendo datos
      select_rcd = """SELECT (C.SECUENCIA + 1) AS ID
                             ,1 ORDEN
                             ,1 AS INDICADOR_1
                             ,1 AS INDICADOR_2
                             ,C.SECUENCIA
                             ,CD.CODIGO_DEUDOR_SBS
                             ,CD.CODIGO_DEUDOR_INTERNO
                             ,CD.CIIU
                             ,CD.ZONA_REGISTRAL
                             ,CD.NUMERO_PARTIDA_REGISTRAL
                             ,CD.OFICINA_REGISTRAL
                             ,CD.TIPO_IDENTIFICACION_REGISTRAL
                             ,CD.TIPO_DOCUMENTO_TRIBUTARIO
                             ,CD.DOCUMENTO_TRIBUTARIO
                             ,CD.TIPO_DOCUMENTO_IDENTIDAD
                             ,CD.DOCUMENTO_IDENTIDAD
                             ,CD.TIPO_PERSONA
                             ,CD.RESIDENCIA
                             ,C.CLASIFICACION_DEUDOR
                             ,CD.MAGNITUD
                             ,CD.ACCIONISTA
                             ,CD.RELACION_LABORAL
                             ,CD.PAIS_RESIDENCIA
                             ,CD.GENERO
                             ,CD.ESTADO_CIVIL
                             ,CD.SIGLA
                             ,CD.APELLIDO_PATERNO_RAZON_SOCIAL
                             ,CD.APELLIDO_MATERNO
                             ,CD.APELLIDO_CASADA
                             ,CD.PRIMER_NOMBRE
                             ,CD.SEGUNDO_NOMBRE
                             ,C.INDICADOR_RIESGO_CAMBIARIO
                             ,C.INDICADOR_ATRASO
                             ,C.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO
                             ,CD.GRUPO_ECONOMICO
                             ,FECHA_NACIMIENTO
                             ,CD.TIPO_DOCUMENTO_IDENTIDAD_COMPLEMENTARIO
                             ,CD.DOCUMENTO_IDENTIDAD_COMPLEMENTARIO
                             ,C.CONDICION_SOBRE_ENDEUDAMIENTO
                             ,CD.CODIGO_DEUDOR_INTERNO AS NUMERO_CUENTA
                             ,CD.RUC
                             ,CD.PFT
                             ,CD.DFT
                             ,CP.MONEDA
                             ,CP.SALDO_CAPITAL
                             ,CP.MONTO_DESEMBOLSO
                             ,CD.FILLER
                             ,CD.TIPO_OPE
                             ,CD.TIP_TRIB
                             ,CD.NUM_TRIB
                             ,CP.TIPO_CLIENTE
                             ,CD.CODIGO_CLIENTE
                         FROM CLIENTE C
                         INNER JOIN CLIENTE_DEUDOR CD ON C.CLIENTE_DEUDOR = CD.ID
                         INNER JOIN CLIENTE_DEUDOR_PRODUCTO CDP ON CD.ID = CDP.CLIENTE_DEUDOR
                         INNER JOIN CLIENTE_PRODUCTO CP ON C.ID = CP.CLIENTE
                         WHERE C.FECHA_PROCESO = '""" + self.fecha_proceso_str + """'
                             AND EXISTS (
                                 SELECT 1
                                 FROM CLIENTE_PRODUCTO_CONCEPTO CPC
                                 WHERE CPC.CLIENTE_PRODUCTO = CP.ID
                                     AND CPC.CUENTA_CONTABLE IS NOT NULL
                                 )
                         ORDER BY SECUENCIA"""
      sql_rcd = pd.read_sql_query(select_rcd, self.connection)
      df_rcd = pd.DataFrame(sql_rcd)
      df_rcd['FECHA_PROCESO'] = self.fecha_proceso_str
      df_rcd['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str  
      
      #Generando formato para el archivo Rcd Cabecera
      df_rcd_format = df_rcd.copy()
      df_rcd_format = df_rcd_format.fillna(' ')
      df_rcd_format['ID'] = df_rcd_format['ID'].astype(str)
      df_rcd_format['ORDEN'] = df_rcd_format['ORDEN'].astype(str)
      df_rcd_format['INDICADOR_1'] = df_rcd_format['INDICADOR_1'].astype(str)
      df_rcd_format['INDICADOR_2'] = df_rcd_format['INDICADOR_2'].astype(str)
      df_rcd_format['SECUENCIA'] = df_rcd_format['SECUENCIA'].astype(str).str.pad(8, side='left', fillchar='0')
      df_rcd_format['CODIGO_DEUDOR_SBS'] = df_rcd_format['CODIGO_DEUDOR_SBS'].astype(str).str.pad(10, side='left', fillchar='0')
      df_rcd_format['CODIGO_DEUDOR_INTERNO'] = df_rcd_format['CODIGO_DEUDOR_INTERNO'].astype(str).str.pad(20, side='right', fillchar=' ')
      #df_rcd_format['CIIU'] = df_rcd_format['CIIU'].fillna('9999')
      df_rcd_format['CIIU'] = np.where(df_rcd_format['CIIU'].str.strip()=='', '9999', df_rcd_format['CIIU'])
      df_rcd_format['CIIU'] = df_rcd_format['CIIU'].astype(str).str.pad(4, side='right', fillchar=' ')
      df_rcd_format['ZONA_REGISTRAL'] = df_rcd_format['ZONA_REGISTRAL'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format['OFICINA_REGISTRAL'] = df_rcd_format['OFICINA_REGISTRAL'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format['TIPO_IDENTIFICACION_REGISTRAL'] = df_rcd_format['TIPO_IDENTIFICACION_REGISTRAL'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['NUMERO_PARTIDA_REGISTRAL'] = df_rcd_format['NUMERO_PARTIDA_REGISTRAL'].astype(str).str.pad(10, side='left', fillchar='0')
      df_rcd_format['TIPO_DOCUMENTO_TRIBUTARIO'] = df_rcd_format['TIPO_DOCUMENTO_TRIBUTARIO'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['DOCUMENTO_TRIBUTARIO'] = df_rcd_format['DOCUMENTO_TRIBUTARIO'].astype(str).str.pad(11, side='left', fillchar=' ')      
      df_rcd_format['TIPO_DOCUMENTO_IDENTIDAD'] = df_rcd_format['TIPO_DOCUMENTO_IDENTIDAD'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['DOCUMENTO_IDENTIDAD'] = df_rcd_format['DOCUMENTO_IDENTIDAD'].astype(str).str.pad(12, side='right', fillchar=' ')
      df_rcd_format['TIPO_PERSONA'] = df_rcd_format['TIPO_PERSONA'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['RESIDENCIA'] = df_rcd_format['RESIDENCIA'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['MAGNITUD'] = df_rcd_format['MAGNITUD'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['ACCIONISTA'] = df_rcd_format['ACCIONISTA'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['RELACION_LABORAL'] = df_rcd_format['RELACION_LABORAL'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['PAIS_RESIDENCIA'] = df_rcd_format['PAIS_RESIDENCIA'].astype(str).str.pad(4, side='right', fillchar=' ')
      df_rcd_format['GENERO'] = df_rcd_format['GENERO'].astype(str).str.pad(1, side='right', fillchar='-')
      df_rcd_format['ESTADO_CIVIL'] = df_rcd_format['ESTADO_CIVIL'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['SIGLA'] = df_rcd_format['SIGLA'].astype(str).str.pad(20, side='right', fillchar=' ')      
      df_rcd_format['APELLIDO_PATERNO_RAZON_SOCIAL'] = df_rcd_format['APELLIDO_PATERNO_RAZON_SOCIAL'].astype(str).str.pad(120, side='right', fillchar=' ')          
      df_rcd_format['APELLIDO_MATERNO'] = df_rcd_format['APELLIDO_MATERNO'].astype(str).str.pad(40, side='right', fillchar=' ')
      df_rcd_format['APELLIDO_CASADA'] = df_rcd_format['APELLIDO_CASADA'].astype(str).str.pad(40, side='right', fillchar=' ')
      df_rcd_format['PRIMER_NOMBRE'] = df_rcd_format['PRIMER_NOMBRE'].astype(str).str.pad(40, side='right', fillchar=' ')
      df_rcd_format['SEGUNDO_NOMBRE'] = df_rcd_format['SEGUNDO_NOMBRE'].astype(str).str.pad(40, side='right', fillchar=' ')
      df_rcd_format['INDICADOR_RIESGO_CAMBIARIO'] = df_rcd_format['INDICADOR_RIESGO_CAMBIARIO'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['INDICADOR_ATRASO'] = df_rcd_format['INDICADOR_ATRASO'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO'] = df_rcd_format['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO'].astype(str).str.pad(5, side='right', fillchar=' ')
      df_rcd_format['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO_2'] = df_rcd_format['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO'].astype(str).str.pad(1, side='right', fillchar=' ')
      df_rcd_format['GRUPO_ECONOMICO'] = df_rcd_format['GRUPO_ECONOMICO'].astype(str).str.pad(20, side='right', fillchar=' ')         
      #f_rcd_format['FECHA_NACIMIENTO'] = pd.to_datetime(df_rcd_format['FECHA_NACIMIENTO']).dt.strftime('%Y%m%d')
      df_rcd_format['FECHA_NACIMIENTO'] = df_rcd_format['FECHA_NACIMIENTO'].astype(str).str.pad(8, side='left', fillchar='0')      
      df_rcd_format['TIPO_DOCUMENTO_IDENTIDAD_COMPLEMENTARIO'] = df_rcd_format['TIPO_DOCUMENTO_IDENTIDAD_COMPLEMENTARIO'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format['DOCUMENTO_IDENTIDAD_COMPLEMENTARIO'] = df_rcd_format['DOCUMENTO_IDENTIDAD_COMPLEMENTARIO'].astype(str).str.pad(40, side='right', fillchar=' ')
      df_rcd_format['CONDICION_SOBRE_ENDEUDAMIENTO'] = df_rcd_format['CONDICION_SOBRE_ENDEUDAMIENTO'].astype(str).str.pad(1, side='right', fillchar=' ')      
      df_rcd_format = df_rcd_format.where(pd.notnull(df_rcd_format), None)
      df_rcd_format = df_rcd_format.fillna('')
      df_rcd_format['LINEA'] = df_rcd_format['INDICADOR_1'] + df_rcd_format['INDICADOR_2'] + df_rcd_format['SECUENCIA'] + df_rcd_format['CODIGO_DEUDOR_SBS'] + df_rcd_format['CODIGO_DEUDOR_INTERNO'] + df_rcd_format['CIIU'] + df_rcd_format['ZONA_REGISTRAL'] + df_rcd_format['OFICINA_REGISTRAL'] + df_rcd_format['TIPO_IDENTIFICACION_REGISTRAL'] + df_rcd_format['NUMERO_PARTIDA_REGISTRAL'] + df_rcd_format['TIPO_DOCUMENTO_TRIBUTARIO'] + df_rcd_format['DOCUMENTO_TRIBUTARIO'] + df_rcd_format['TIPO_DOCUMENTO_IDENTIDAD'] + df_rcd_format['DOCUMENTO_IDENTIDAD'] + df_rcd_format['TIPO_PERSONA'] + df_rcd_format['RESIDENCIA'] + df_rcd_format['MAGNITUD'] + df_rcd_format['ACCIONISTA'] + df_rcd_format['RELACION_LABORAL'] + df_rcd_format['PAIS_RESIDENCIA'] + df_rcd_format['GENERO'] + df_rcd_format['ESTADO_CIVIL'] + df_rcd_format['SIGLA'] + df_rcd_format['APELLIDO_PATERNO_RAZON_SOCIAL'] + df_rcd_format['APELLIDO_MATERNO'] + df_rcd_format['APELLIDO_CASADA'] + df_rcd_format['PRIMER_NOMBRE'] + df_rcd_format['SEGUNDO_NOMBRE'] + df_rcd_format['INDICADOR_RIESGO_CAMBIARIO'] + df_rcd_format['INDICADOR_ATRASO'] + df_rcd_format['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO'] + df_rcd_format['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO_2'] + df_rcd_format['GRUPO_ECONOMICO'] + df_rcd_format['FECHA_NACIMIENTO'] + df_rcd_format['TIPO_DOCUMENTO_IDENTIDAD_COMPLEMENTARIO'] + df_rcd_format['DOCUMENTO_IDENTIDAD_COMPLEMENTARIO'] + df_rcd_format['CONDICION_SOBRE_ENDEUDAMIENTO']
      
      #Base de datos
      df_rcd_format_db = df_rcd_format[['FECHA_PROCESO','ID', 'ORDEN', 'LINEA']]
      #display(df_rcd_format_db) 
          
      #Filtrando datos para BF_CARGA_TMP_RCD_1 - Modificar según campos a insertar
      df_bf_carga_tmp_rcd1 = df_rcd.filter(['FECHA_PROCESO','CODIGO_DEUDOR_SBS','APELLIDO_PATERNO_RAZON_SOCIAL','SIGLA','CIIU','TIPO_DOCUMENTO_TRIBUTARIO','DOCUMENTO_TRIBUTARIO','TIPO_DOCUMENTO_IDENTIDAD','DOCUMENTO_IDENTIDAD','RESIDENCIA','MAGNITUD','ACCIONISTA','PAIS_RESIDENCIA','GENERO','ESTADO_CIVIL','FECHA_NACIMIENTO','TIPO_DOCUMENTO_IDENTIDAD_COMPLEMENTARIO','DOCUMENTO_IDENTIDAD_COMPLEMENTARIO','NUMERO_CUENTA','FECHA_ACTUALIZACION',
      'ZONA_REGISTRAL','RUC','PFT','DFT','SALDO_PRINCIPAL','MONTO_DESEMBOLSO','MONEDA','FILLER','TIPO_OPE','TIP_TRIB','NUM_TRIB','TIPO_CLIENTE','CODIGO_CLIENTE'])
      df_bf_carga_tmp_rcd1['NUMERO_TARJETA'] = df_bf_carga_tmp_rcd1['NUMERO_CUENTA']

      #Limpiando la tabla
      delete_bf_carga_tmp_rcd1 = "DELETE FROM " + self.tabla_destino + " WHERE FECHA_PROCESO = '" + self.fecha_proceso_str + "'"
      self.cursor.execute(delete_bf_carga_tmp_rcd1)
      self.connection.commit()
      
      #Insertando datos
      insert_bf_carga_tmp_rcd1 = "INSERT INTO [dbo].[" + self.tabla_destino + "] (FECHA_PROCESO,NUMERO_CUENTA,CODIGOSBS,NUMERO_TARJETA,NOMBRE,SIGLA,CODIGOCIUU,REGPUBLICO,RUC,PFT,TIPODOCUMENTO,NUMDOCUMENTO,RESIDENCIA,DFT,MONEDA,SALDO,MONTOCREDITO,TIPODOCTRIBUT,NUMDOCTRIBUT,FILLER,TIPOPE,MAGNITUDEMP,ACCIONISTA,TIPTRIB,NUMTRIB, SEXO,ESTCIVIL,TIPOCLIENTE, PAISRESIDENCIA,FECHANAC,TIPODOCIDENTCOMP,DOCIDENTCOMP,FECHA_ACTUALIZACION) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,',?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df_bf_carga_tmp_rcd1.values.tolist()
      if values:        
        self.cursor.executemany(insert_bf_carga_tmp_rcd1, df_bf_carga_tmp_rcd1.values.tolist())
        self.connection.commit()       
      
      ###### ------ Rcd detalle ------ ######
      #Obteniendo datos
      select_rcd_detalle = """ SELECT (C.SECUENCIA + 1) AS ID
                                     ,2 ORDEN
                                     ,'1' AS INDICADOR_1
                                     ,'2' AS INDICADOR_2
                                     ,C.SECUENCIA
                                     ,O.ID AS ID_OFICINA
                                     ,O.UBIGEO
                                     ,LEFT(O.UBIGEO, 2) AS DEPARTAMENTO
                                     ,SUBSTRING(O.UBIGEO, 3, 4) AS PROVINCIA
                                     ,RIGHT(O.UBIGEO, 2) AS DISTRITO
                                     ,CP.TIPO_CREDITO_SBS
                                     ,SUM(IMPORTE) AS IMPORTE
                                     ,CPC.CUENTA_CONTABLE
                                     ,CP.DIAS_ATRASO
                                     ,CD.CODIGO_DEUDOR_INTERNO AS NUMERO_CUENTA
                                     ,P.ATTRIBUTE1 AS PRODUCTO
                                     ,CD.CODIGO_CLIENTE
                                     ,E.CODIGO_EMPRESA_SBS AS EMPRESA
                                     ,CDP.NUMERO_CUENTA AS COD_OPORACION
                                     ,CDP.ESTADO_CUENTA
                                 FROM CLIENTE C
                                 JOIN CLIENTE_DEUDOR CD ON C.CLIENTE_DEUDOR = CD.ID
                                 JOIN CLIENTE_DEUDOR_PRODUCTO CDP ON CD.ID = CDP.CLIENTE_DEUDOR
                                 JOIN CLIENTE_PRODUCTO CP ON C.ID = CP.CLIENTE
                                     AND CDP.ID = CP.CLIENTE_DEUDOR_PRODUCTO
                                 JOIN CLIENTE_PRODUCTO_CONCEPTO CPC ON CP.ID = CPC.CLIENTE_PRODUCTO
                                 JOIN AGENCIA A ON CD.AGENCIA = A.ID
                                 JOIN AGENCIA_OFICINA AO ON AO.AGENCIA = A.ID
                                 JOIN OFICINA O ON AO.OFICINA = O.ID
                                 JOIN EMPRESA E ON E.ID = CD.EMPRESA
                                 JOIN PRODUCTO P ON P.ID = CDP.PRODUCTO
                                 WHERE C.FECHA_PROCESO = '""" + self.fecha_proceso_str + """'
                                 GROUP BY C.ID
                                     ,O.ID
                                     ,C.SECUENCIA
                                     ,O.UBIGEO
                                     ,CP.TIPO_CREDITO_SBS
                                     ,CPC.CUENTA_CONTABLE
                                     ,CP.DIAS_ATRASO
                                     ,CD.CODIGO_DEUDOR_INTERNO
									 ,E.CODIGO_EMPRESA_SBS
									 ,P.ATTRIBUTE1
									 ,CD.CODIGO_CLIENTE
									 ,CDP.NUMERO_CUENTA
									 ,CDP.ESTADO_CUENTA
									 ORDER BY SECUENCIA"""      
      sql_rcd_detalle = pd.read_sql_query(select_rcd_detalle, self.connection)
      df_rcd_detalle = pd.DataFrame(sql_rcd_detalle)
      df_rcd_detalle.rename({'CUENTA_CONTABLE': 'CUENTA_CONTABLE_RCD_DETALLE'}, axis=1, inplace=True)

      #Cuenta contable vigente
      df_cuenta_contable_vigente = PRC_RCD_PROVISIONES(1).cuentaContableVigente(self.fecha_proceso_str)
      df_cuenta_contable_vigente.rename({'ID': 'CUENTA_CONTABLE_RCD_DETALLE'}, axis=1, inplace=True)
            
      #Rcd detalle join Cuenta contable vigente
      df_rcd_detalle = pd.merge(df_rcd_detalle, df_cuenta_contable_vigente, on='CUENTA_CONTABLE_RCD_DETALLE', how='inner')
      df_rcd_detalle['CONDICION_DIAS'] = np.where(df_rcd_detalle['CONDICION_DIAS'], df_rcd_detalle['DIAS_ATRASO'], 0)
      df_rcd_detalle.loc[df_rcd_detalle['INFORME_SBS'] == 1]      
      df_rcd_detalle.groupby(['ID','ID_OFICINA','SECUENCIA', 'UBIGEO','CUENTA_CONTABLE','TIPO_CREDITO_SBS_x','CONDICION_DIAS','CONDICION_ESPECIAL', 'CONDICION_DISPONIBILIDAD', 'FACTOR_CONVERSION_CREDITO'])['IMPORTE'].sum()
      df_rcd_detalle.rename({'TIPO_CREDITO_SBS_x': 'TIPO_CREDITO_SBS'}, axis=1, inplace=True)      
      df_rcd_detalle['FECHA_PROCESO'] = self.fecha_proceso_str
      df_rcd_detalle['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      
      #Generando formato para el archivo Rcd Detalle
      df_rcd_format_detalle = df_rcd_detalle.copy()
      df_rcd_format_detalle = df_rcd_format_detalle.fillna(' ')
      df_rcd_format_detalle['ID'] = df_rcd_format_detalle['ID'].astype(str)
      df_rcd_format_detalle['ORDEN'] = df_rcd_format_detalle['ORDEN'].astype(str)
      df_rcd_format_detalle['INDICADOR_1'] = df_rcd_format_detalle['INDICADOR_1'].astype(str)
      df_rcd_format_detalle['INDICADOR_2'] = df_rcd_format_detalle['INDICADOR_2'].astype(str)
      df_rcd_format_detalle['SECUENCIA'] = df_rcd_format_detalle['SECUENCIA'].astype(str).str.pad(8, side='left', fillchar='0')
      df_rcd_format_detalle['ID_OFICINA'] = df_rcd_format_detalle['ID_OFICINA'].astype(str).str.pad(4, side='left', fillchar='0')      
      df_rcd_format_detalle['DEPARTAMENTO'] = df_rcd_format_detalle['DEPARTAMENTO'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format_detalle['PROVINCIA'] = df_rcd_format_detalle['PROVINCIA'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format_detalle['DISTRITO'] = df_rcd_format_detalle['DISTRITO'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format_detalle['CUENTA_CONTABLE'] = df_rcd_format_detalle['CUENTA_CONTABLE'].astype(str).str.pad(14, side='right', fillchar='0')
      df_rcd_format_detalle['TIPO_CREDITO_SBS'] = df_rcd_format_detalle['TIPO_CREDITO_SBS'].astype(str).str.pad(2, side='right', fillchar=' ')      
      df_rcd_format_detalle['IMPORTE'] = (df_rcd_format_detalle['IMPORTE'].round(decimals=2)*(100)).astype(int)      
      df_rcd_format_detalle['IMPORTE'] = df_rcd_format_detalle['IMPORTE'].astype(str).str.pad(18, side='left', fillchar='0')
      df_rcd_format_detalle['CONDICION_DIAS'] = df_rcd_format_detalle['CONDICION_DIAS'].astype(str).str.pad(4, side='left', fillchar='0')      
      df_rcd_format_detalle['CONDICION_ESPECIAL'] = df_rcd_format_detalle['CONDICION_ESPECIAL'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format_detalle['CONDICION_DISPONIBILIDAD'] = df_rcd_format_detalle['CONDICION_DISPONIBILIDAD'].astype(str).str.pad(2, side='right', fillchar=' ')      
      df_rcd_format_detalle['FACTOR_CONVERSION_CREDITO'] = df_rcd_format_detalle['FACTOR_CONVERSION_CREDITO'].astype(str).str.pad(2, side='right', fillchar=' ')  
      df_rcd_format_detalle = df_rcd_format_detalle.where(pd.notnull(df_rcd_format_detalle), None)
      df_rcd_format_detalle = df_rcd_format_detalle.fillna('')
      df_rcd_format_detalle['LINEA'] = df_rcd_format_detalle['INDICADOR_1'] + df_rcd_format_detalle['INDICADOR_2'] + df_rcd_format_detalle['SECUENCIA'] + df_rcd_format_detalle['ID_OFICINA'] + df_rcd_format_detalle['DEPARTAMENTO'] + df_rcd_format_detalle['PROVINCIA'] + df_rcd_format_detalle['DISTRITO'] + df_rcd_format_detalle['CUENTA_CONTABLE'] + df_rcd_format_detalle['TIPO_CREDITO_SBS'] + df_rcd_format_detalle['IMPORTE'] + df_rcd_format_detalle['CONDICION_DIAS'] + df_rcd_format_detalle['CONDICION_ESPECIAL'] + df_rcd_format_detalle['CONDICION_DISPONIBILIDAD'] + df_rcd_format_detalle['FACTOR_CONVERSION_CREDITO']
           
      #Base de datos
      df_rcd_format_detalle_db = df_rcd_format_detalle[['FECHA_PROCESO','ID', 'ORDEN', 'LINEA']]
      #display(df_rcd_format_detalle_db) 
            
      #Filtrando datos para BF_CARGA_TMP_RCD_2 - Modificar según campos a insertar
      df_bf_carga_tmp_rcd2 = df_rcd_detalle.filter(['FECHA_PROCESO','NUMERO_CUENTA','PRODUCTO','CODIGO_CLIENTE','TIPO_CREDITO_SBS','IMPORTE','DIAS_ATRASO','EMPRESA','COD_OPERACION','ID_OFICINA','UBIGUEO','ESTADO_CUENTA','CUENTA_CONTABLE','FECHA_ACTUALIZACION'])
      
      #Limpiando la tabla
      delete_bf_carga_tmp_rcd2 = "DELETE FROM " + self.tabla_destino_2 + " WHERE FECHA_PROCESO = '" + self.fecha_proceso_str + "'"
      self.cursor.execute(delete_bf_carga_tmp_rcd2)
      self.connection.commit()
      
      #Insertando datos
      insert_bf_carga_tmp_rcd2 = "INSERT INTO [dbo].[" + self.tabla_destino_2 + "] (FECHA_PROCESO,NROCUENTACMR,CODIGOPRODUCTO,CODIGOCLIENTE,TIPOCREDITO,SALDONORMAL,DIASMORA,IDCARTERA,NROOPERACION,CODOFIEMP,UBIGEOOFIEMP,FLAGESTACTA,CODCTACONTABLE,FECHA_ACTUALIZACION) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df_bf_carga_tmp_rcd2.values.tolist()
      if values:
        self.cursor.executemany(insert_bf_carga_tmp_rcd2, values)
        self.connection.commit()  
            
      ###### ------ Rcd totales ------ ######
      #Obteniendo ultima secuencia
      select_ultima_secuencia = ("SELECT ISNULL((MAX(SECUENCIA) + 1), 0)  AS MAX_SECUENCIA FROM CLIENTE WHERE FECHA_PROCESO = '" + self.fecha_proceso_str + "'")
      sql_ultima_secuencia = pd.read_sql_query(select_ultima_secuencia, self.connection)
      df_ultima_secuencia = pd.DataFrame(sql_ultima_secuencia)
      ultima_secuencia = df_ultima_secuencia.loc[0, 'MAX_SECUENCIA']
      ultima_secuencia_str = str(ultima_secuencia)
      
      #Obteniendo datos
      select_rcd_totales = """SELECT '""" + self.fecha_proceso_str + """' AS FECHA_PROCESO
                                            ,""" + ultima_secuencia_str + """ + 1 AS ID
                                            ,2 ORDEN
                                            ,*
                                        FROM (
                                            SELECT '2' AS INDICADOR_1,
                                                   '2' AS INDICADOR_2,
                                                   """ + ultima_secuencia_str + """ AS SECUENCIA
                                                   ,O.ID AS ID_OFICINA
                                                   ,O.UBIGEO
                                                   ,LEFT(O.UBIGEO, 2) AS DEPARTAMENTO
                                                   ,SUBSTRING(O.UBIGEO, 3, 4) AS PROVINCIA
                                                   ,RIGHT(O.UBIGEO, 2) AS DISTRITO
                                                   ,CP.TIPO_CREDITO_SBS
                                                   ,SUM(IMPORTE) AS IMPORTE
                                                   ,CPC.CUENTA_CONTABLE
                                                   ,CP.DIAS_ATRASO
                                               FROM CLIENTE C
                                               JOIN CLIENTE_DEUDOR CD ON C.CLIENTE_DEUDOR = CD.ID
                                               JOIN CLIENTE_DEUDOR_PRODUCTO CDP ON CD.ID = CDP.CLIENTE_DEUDOR
                                               JOIN CLIENTE_PRODUCTO CP ON C.ID = CP.CLIENTE
                                                   AND CDP.ID = CP.CLIENTE_DEUDOR_PRODUCTO
                                               JOIN CLIENTE_PRODUCTO_CONCEPTO CPC ON CP.ID = CPC.CLIENTE_PRODUCTO
                                               JOIN AGENCIA A ON CD.AGENCIA = A.ID
                                               JOIN AGENCIA_OFICINA AO ON AO.AGENCIA = A.ID
                                               JOIN OFICINA O ON AO.OFICINA = O.ID
                                               WHERE C.FECHA_PROCESO = '""" + self.fecha_proceso_str + """'
                                               GROUP BY C.ID
                                                   ,O.ID
                                                   ,O.UBIGEO
                                                   ,CP.TIPO_CREDITO_SBS
                                                   ,CPC.CUENTA_CONTABLE
                                                   ,CP.DIAS_ATRASO
                                               ) B"""      

      sql_rcd_totales = pd.read_sql_query(select_rcd_totales, self.connection)
      df_rcd_totales = pd.DataFrame(sql_rcd_totales)
      df_rcd_totales.rename({'CUENTA_CONTABLE': 'CUENTA_CONTABLE_RCD_TOTALES'}, axis=1, inplace=True)

      #Cuenta contable vigente
      df_cuenta_contable_vigente.rename({'CUENTA_CONTABLE_RCD_DETALLE': 'CUENTA_CONTABLE_RCD_TOTALES'}, axis=1, inplace=True)
            
      #Rcd detalle join Cuenta contable vigente
      df_rcd_totales = pd.merge(df_rcd_totales, df_cuenta_contable_vigente, on='CUENTA_CONTABLE_RCD_TOTALES', how='inner')
      df_rcd_totales['CONDICION_DIAS'] = np.where(df_rcd_totales['CONDICION_DIAS'], df_rcd_totales['DIAS_ATRASO'], 0)
      df_rcd_totales.loc[df_rcd_totales['INFORME_SBS'] == 1]
      
      df_rcd_totales.groupby(['ID','ID_OFICINA','UBIGEO','CUENTA_CONTABLE','TIPO_CREDITO_SBS_x','CONDICION_DIAS','CONDICION_ESPECIAL', 'CONDICION_DISPONIBILIDAD', 'FACTOR_CONVERSION_CREDITO'])['IMPORTE'].sum()
      df_rcd_totales.rename({'TIPO_CREDITO_SBS_x': 'TIPO_CREDITO_SBS'}, axis=1, inplace=True)
      
      #Generando formato para el archivo Rcd Totales
      df_rcd_format_totales = df_rcd_totales.copy()
      df_rcd_format_totales = df_rcd_totales.fillna(' ')
      df_rcd_format_totales['ID'] = df_rcd_format_totales['ID'].astype(str)
      df_rcd_format_totales['ORDEN'] = df_rcd_format_totales['ORDEN'].astype(str)
      df_rcd_format_totales['INDICADOR_1'] = df_rcd_format_totales['INDICADOR_1'].astype(str)
      df_rcd_format_totales['INDICADOR_2'] = df_rcd_format_totales['INDICADOR_2'].astype(str)
      df_rcd_format_totales['SECUENCIA'] = df_rcd_format_totales['SECUENCIA'].astype(str).str.pad(8, side='left', fillchar='0')
      df_rcd_format_totales['ID_OFICINA'] = df_rcd_format_totales['ID_OFICINA'].astype(str).str.pad(4, side='left', fillchar=' ')      
      df_rcd_format_totales['DEPARTAMENTO'] = df_rcd_format_totales['DEPARTAMENTO'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format_totales['PROVINCIA'] = df_rcd_format_totales['PROVINCIA'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format_totales['DISTRITO'] = df_rcd_format_totales['DISTRITO'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format_totales['CUENTA_CONTABLE'] = df_rcd_format_totales['CUENTA_CONTABLE'].astype(str).str.pad(14, side='right', fillchar='0')
      df_rcd_format_totales['TIPO_CREDITO_SBS'] = df_rcd_format_totales['TIPO_CREDITO_SBS'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format_totales['IMPORTE'] = (df_rcd_format_totales['IMPORTE'].round(decimals=2)*(100)).astype(int)
      df_rcd_format_totales['IMPORTE'] = df_rcd_format_totales['IMPORTE'].astype(str).str.pad(18, side='left', fillchar=' ')
      df_rcd_format_totales['CONDICION_DIAS'] = df_rcd_format_totales['CONDICION_DIAS'].astype(str).str.pad(4, side='left', fillchar='0')      
      df_rcd_format_totales['CONDICION_ESPECIAL'] = df_rcd_format_totales['CONDICION_ESPECIAL'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format_totales['CONDICION_DISPONIBILIDAD'] = df_rcd_format_totales['CONDICION_DISPONIBILIDAD'].astype(str).str.pad(2, side='right', fillchar=' ')      
      df_rcd_format_totales['FACTOR_CONVERSION_CREDITO'] = df_rcd_format_totales['FACTOR_CONVERSION_CREDITO'].astype(str).str.pad(2, side='right', fillchar=' ')
      df_rcd_format_totales = df_rcd_format_totales.where(pd.notnull(df_rcd_format_totales), None)
      df_rcd_format_totales = df_rcd_format_totales.fillna('')
      df_rcd_format_totales['LINEA'] = df_rcd_format_totales['INDICADOR_1'] + df_rcd_format_totales['INDICADOR_2'] + df_rcd_format_totales['SECUENCIA'] + df_rcd_format_totales['ID_OFICINA'] + df_rcd_format_totales['DEPARTAMENTO'] + df_rcd_format_totales['PROVINCIA'] + df_rcd_format_totales['DISTRITO'] + df_rcd_format_totales['CUENTA_CONTABLE'] + df_rcd_format_totales['TIPO_CREDITO_SBS'] + df_rcd_format_totales['IMPORTE'] + df_rcd_format_totales['CONDICION_DIAS'] + df_rcd_format_totales['CONDICION_ESPECIAL'] + df_rcd_format_totales['CONDICION_DISPONIBILIDAD'] + df_rcd_format_totales['FACTOR_CONVERSION_CREDITO']   
      
      #Base de datos
      df_rcd_format_totales_db = df_rcd_format_totales[['FECHA_PROCESO','ID', 'ORDEN', 'LINEA']]
      
      #Uniendo Rcd cabecera, Rcd detalle, y Rcd totales
      df_cabecera_detalle_totales_db = pd.concat([df_rcd_format_db, df_rcd_format_detalle_db, df_rcd_format_totales_db], ignore_index = True)
      #display(df_cabecera_detalle_totales_db) 
      
      #Empresa
      select_codigo_empresa = "SELECT CODIGO_EMPRESA_SBS FROM EMPRESA WHERE ID = "  + self.empresa_str    
      sql_codigo_empresa = pd.read_sql_query(select_codigo_empresa, self.connection)
      df_codigo_empresa = pd.DataFrame(sql_codigo_empresa)
      codigo_empresa = df_codigo_empresa.loc[0, 'CODIGO_EMPRESA_SBS']
      
      #Agregando títulos
      titulo_rcd_cabecera = {'FECHA_PROCESO':self.fecha_proceso_str,'ID':'1', 'ORDEN':'0', 'LINEA':'0106'+'01'+'00'+codigo_empresa+self.fecha_proceso.strftime('%Y%m%d')+'012'+'               '+'000000000000000'}
      titulo_rcd_detalle = {'FECHA_PROCESO':self.fecha_proceso_str,'ID':str(ultima_secuencia + 1), 'ORDEN':'1', 'LINEA':'21'+ultima_secuencia_str.zfill(8)+ultima_secuencia_str}      
      df_cabecera_detalle_totales_db = df_cabecera_detalle_totales_db.append(titulo_rcd_cabecera, ignore_index=True)
      df_cabecera_detalle_totales_db = df_cabecera_detalle_totales_db.append(titulo_rcd_detalle, ignore_index=True)      
      df_cabecera_detalle_totales_db = df_cabecera_detalle_totales_db.sort_values(by=['ID','ORDEN'])
      
      #Limpiando la tabla
      delete_rcd = "DELETE FROM [dbo].[RCD] WHERE FECHA_PROCESO = '" + self.fecha_proceso_str + "'"
      self.cursor.execute(delete_rcd)
      self.connection.commit()
      
      #Insertando datos
      insert_cabecera_detalle_totales = "INSERT INTO [dbo].[RCD](FECHA_PROCESO,N,O,LINEA) VALUES (?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df_cabecera_detalle_totales_db.values.tolist()
      if values:
        self.cursor.executemany(insert_cabecera_detalle_totales, values)
        self.connection.commit()  
      self.cursor.close() 
      
      #Agregando títulos
      df_cabecera_detalle_totales_file = df_cabecera_detalle_totales_db[['LINEA']]
      np.savetxt(self.ruta_destino + 'RCD' + self.fecha_proceso_str[:4] + self.fecha_proceso_str[5:7] + "." + codigo_empresa, df_cabecera_detalle_totales_file.values, fmt='%s')
            
#Ejecucion prueba
objeto = PrcGeneraPlanoRcd(4)
objeto.generarPlanoRcd()